In [26]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import tensorflow as tf
import pandas as pd
import numpy as np
import pickle
import glob

In [27]:
# 그냥 간편하게 사용하려고 만든 함수인듯
def load_puck_model(model_path):
    return tf.keras.models.load_model(model_path)

In [28]:
# MinMax 정규화

def standardize_data(df, col, min_value, max_value):
    #global scaler
    scaler = MinMaxScaler()
    
    # 표준화할 데이터 준비 (예시로 특정 열만 사용)
    df.reset_index(drop=True, inplace=True)
    data_to_standardize = df.loc[:,[col]]

    # min_value와 max_value를 Series로 변환
    min_series = pd.Series([min_value], index=[len(data_to_standardize)])
    max_series = pd.Series([max_value], index=[len(data_to_standardize) + 1])

    # 최대값, 최솟값 삽입 (Series를 DataFrame에 추가)
    data_to_standardize.loc[len(data_to_standardize)] = min_value
    data_to_standardize.loc[len(data_to_standardize)] = max_value
    
    # 데이터를 표준화
    standardized_data = scaler.fit_transform(data_to_standardize)

    # 표준화된 데이터를 DataFrame으로 변환
    standardized_df = pd.DataFrame(standardized_data, columns=[col])

    # 원래 데이터에 표준화된 데이터로 업데이트
    df.loc[:, [col]] = standardized_df[[col]].astype(float)

    return df[[col]]

In [29]:
# Acc_X, Acc_Y, Gyr_Z 추출하는 함수
def extract_data(input_file):
    file = pd.read_csv(input_file)
    file.drop(['Time', 'Acc_Z', ' Gyr_X', ' Gyr_Y'], axis=1, inplace=True)
    file = file.drop(0, axis=0)
    file.reset_index(drop=True, inplace=True)
    file = file.astype(float)
    return file

In [30]:
# 구간을 자르는 함수
def cut_length(input_file):
    normal_data = input_file.copy()

    for col in normal_data.columns:
        if col == "Acc_X":
            normal_data[col] = standardize_data(normal_data, col, -4, 4)
        elif col == "Acc_Y":
            normal_data[col] = standardize_data(normal_data, col, -4, 4)
        else:
            normal_data[col] = standardize_data(normal_data, col, -2000, 2000)
        
    for j in range(1, len(input_file)):
        if abs(float(input_file.iloc[j]['Acc_X'])) > 0.1:
            insert_data = normal_data[j:j+10]
            break
    return insert_data.values

In [31]:
def puck_predict(input_file):
    model = load_puck_model('path106')
    # 예측 시 스케일러 불러오기
    with open('scaler.pkl', 'rb') as f:
        scaler = pickle.load(f)
    # 데이터 전처리
    data = []
    
    file = extract_data(input_file)
    insert_data = cut_length(file)
    data.append(insert_data)
    data = np.array(data)
    
    score = model.predict(data)
    result = scaler.inverse_transform(score)

    return result

In [36]:
# 파일 전부 읽어오기

path = "D:\ㅎㅅㅂ\Puck_data"

file_list = glob.glob(path+'/**')
files = []
for i in range(len(file_list)):
    f = glob.glob(file_list[i]+'/**')
    for x in f:
        if x.endswith('.csv'):
            files.append(x)
input_file = files[len(files)-1]
input_file

'D:\\ㅎㅅㅂ\\Puck_data\\Assets\\SensorData_20250115_180023(95,308,64).csv'

In [37]:
puck_predict(input_file)

1/1 [==============================] - 0s 87ms/step


array([[181.94423, 315.31696]], dtype=float32)